# Assignment 4 - CNN

Previously in 2_fullyconnected.ipynb and 3_regularization.ipynb (also 2 more assignments part of 3_regularization.ipynb), we trained fully connected networks to classify notMNIST characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:

* convolutions need the image data formatted as a cube (width by height by #channels)
* labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [9]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(    
            [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
    
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [10]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 3.178333
Minibatch accuracy: 25.0%
Validation accuracy: 11.9%
Minibatch loss at step 50: 1.413656
Minibatch accuracy: 37.5%
Validation accuracy: 51.8%
Minibatch loss at step 100: 0.852075
Minibatch accuracy: 75.0%
Validation accuracy: 70.2%
Minibatch loss at step 150: 0.518346
Minibatch accuracy: 87.5%
Validation accuracy: 76.3%
Minibatch loss at step 200: 1.008632
Minibatch accuracy: 75.0%
Validation accuracy: 76.7%
Minibatch loss at step 250: 1.155398
Minibatch accuracy: 62.5%
Validation accuracy: 77.2%
Minibatch loss at step 300: 0.364634
Minibatch accuracy: 87.5%
Validation accuracy: 78.8%
Minibatch loss at step 350: 0.684691
Minibatch accuracy: 87.5%
Validation accuracy: 78.0%
Minibatch loss at step 400: 0.225201
Minibatch accuracy: 100.0%
Validation accuracy: 79.6%
Minibatch loss at step 450: 0.837756
Minibatch accuracy: 87.5%
Validation accuracy: 77.9%
Minibatch loss at step 500: 0.859082
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%


## Problem 1

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

In [11]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(    
            [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        hidden = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        # data ---> conv2d | layer1 ---> relu --> max_pool
        #
        # data input tensor:      [16, 28, 28, 1]     4D tensor: (batch, h, w, ch)
        # layer1 filter tensor:   [5, 5, 1, 16]
        # conv2d output tensor:   [16, 28, 28, 16]
        # relu output tensor:     [16, 28, 28, 16]
        # max_pool output tensor: [16, 14, 14 16]
        
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        hidden = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2 ,2, 1], padding='SAME')
        # hidden ---> conv2d | layer2 ---> relu --> max_pool
        #
        # hidden input tensor:    [16, 14, 14, 16]
        # layer2 filter tensor:   [5, 5, 16, 16]
        # conv2d output tensor:   [16, 14, 14, 16]
        # relu output tensor:     [16, 14, 14, 16]
        # max_pool output tensor: [16, 7, 7, 16]
        
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
    
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [13]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%\n' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.108681
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%

Minibatch loss at step 50: 1.858853
Minibatch accuracy: 31.2%
Validation accuracy: 40.4%

Minibatch loss at step 100: 1.461992
Minibatch accuracy: 50.0%
Validation accuracy: 64.5%

Minibatch loss at step 150: 0.496684
Minibatch accuracy: 87.5%
Validation accuracy: 72.8%

Minibatch loss at step 200: 1.049004
Minibatch accuracy: 62.5%
Validation accuracy: 76.3%

Minibatch loss at step 250: 1.479828
Minibatch accuracy: 56.2%
Validation accuracy: 76.3%

Minibatch loss at step 300: 0.396371
Minibatch accuracy: 93.8%
Validation accuracy: 79.7%

Minibatch loss at step 350: 0.520083
Minibatch accuracy: 93.8%
Validation accuracy: 78.8%

Minibatch loss at step 400: 0.209186
Minibatch accuracy: 93.8%
Validation accuracy: 81.0%

Minibatch loss at step 450: 0.768083
Minibatch accuracy: 87.5%
Validation accuracy: 79.8%

Minibatch loss at step 500: 0.684450
Minibatch accuracy: 87.5%
Validation accurac

#### The accuracy of above classifier can be increased with more epochs.

## Problem 2

Try to get the best performance you can using a convolutional net. Look for example at the classic LeNet5 architecture, adding Dropout, and/or adding learning rate decay.

## LeNet-5

In [5]:
import time
import sys

batch_size=64
PIXEL_DEPTH = 255
VALIDATION_SIZE = 5000  # Size of the validation set.
SEED = 66478  # Set to None for random seed.
NUM_EPOCHS = 10
EVAL_BATCH_SIZE = 64
EVAL_FREQUENCY = 100  # Number of steps between evaluations.

def lenet(argv=None):  # pylint: disable=unused-argument
    train_size = train_labels.shape[0]

    # This is where training samples and labels are fed to the graph.
    # These placeholder nodes will be fed a batch of training data at each
    # training step using the {feed_dict} argument to the Run() call below.
    train_data_node = tf.placeholder(
            tf.float32,
            shape=(batch_size, image_size, image_size, num_channels))
    train_labels_node = tf.placeholder(
                                    tf.float32,
                                    shape=(batch_size, num_labels))
    eval_data = tf.placeholder(
                            tf.float32,
                            shape=(EVAL_BATCH_SIZE, image_size, image_size, num_channels))

    # The variables below hold all the trainable weights. They are passed an
    # initial value which will be assigned when when we call:
    # {tf.initialize_all_variables().run()}
    conv1_weights = tf.Variable(
            tf.truncated_normal([5, 5, num_channels, 32],  # 5x5 filter, depth 32.
                                                    stddev=0.1,
                                                    seed=SEED))
    conv1_biases = tf.Variable(tf.zeros([32]))
    conv2_weights = tf.Variable(
            tf.truncated_normal([5, 5, 32, 64],
                                                    stddev=0.1,
                                                    seed=SEED))
    conv2_biases = tf.Variable(tf.constant(0.1, shape=[64]))
    fc1_weights = tf.Variable(  # fully connected, depth 512.
            tf.truncated_normal(
                    [image_size // 4 * image_size // 4 * 64, 512],
                    stddev=0.1,
                    seed=SEED))
    fc1_biases = tf.Variable(tf.constant(0.1, shape=[512]))
    fc2_weights = tf.Variable(
            tf.truncated_normal([512, num_labels],
                                                    stddev=0.1,
                                                    seed=SEED))
    fc2_biases = tf.Variable(tf.constant(0.1, shape=[num_labels]))

    # We will replicate the model structure for the training subgraph, as well
    # as the evaluation subgraphs, while sharing the trainable parameters.
    def model(data, train=False):
        """The Model definition."""
        # 2D convolution, with 'SAME' padding (i.e. the output feature map has
        # the same size as the input). Note that {strides} is a 4D array whose
        # shape matches the data layout: [image index, y, x, depth].
        conv = tf.nn.conv2d(data,
                                                conv1_weights,
                                                strides=[1, 1, 1, 1],
                                                padding='SAME')
        # Bias and rectified linear non-linearity.
        relu = tf.nn.relu(tf.nn.bias_add(conv, conv1_biases))
        # Max pooling. The kernel size spec {ksize} also follows the layout of
        # the data. Here we have a pooling window of 2, and a stride of 2.
        pool = tf.nn.max_pool(relu,
                                                    ksize=[1, 2, 2, 1],
                                                    strides=[1, 2, 2, 1],
                                                    padding='SAME')
        conv = tf.nn.conv2d(pool,
                                                conv2_weights,
                                                strides=[1, 1, 1, 1],
                                                padding='SAME')
        relu = tf.nn.relu(tf.nn.bias_add(conv, conv2_biases))
        pool = tf.nn.max_pool(relu,
                                                    ksize=[1, 2, 2, 1],
                                                    strides=[1, 2, 2, 1],
                                                    padding='SAME')
        # Reshape the feature map cuboid into a 2D matrix to feed it to the
        # fully connected layers.
        pool_shape = pool.get_shape().as_list()
        reshape = tf.reshape(
                pool,
                [pool_shape[0], pool_shape[1] * pool_shape[2] * pool_shape[3]])
        # Fully connected layer. Note that the '+' operation automatically
        # broadcasts the biases.
        hidden = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
        # Add a 50% dropout during training only. Dropout also scales
        # activations such that no rescaling is needed at evaluation time.
        if train:
            hidden = tf.nn.dropout(hidden, 0.5, seed=SEED)
        return tf.matmul(hidden, fc2_weights) + fc2_biases

    # Training computation: logits + cross-entropy loss.
    logits = model(train_data_node, True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            logits, train_labels_node))

    # L2 regularization for the fully connected parameters.
    regularizers = (tf.nn.l2_loss(fc1_weights) + tf.nn.l2_loss(fc1_biases) +
                                    tf.nn.l2_loss(fc2_weights) + tf.nn.l2_loss(fc2_biases))
    # Add the regularization term to the loss.
    loss += 5e-4 * regularizers

    # Optimizer: set up a variable that's incremented once per batch and
    # controls the learning rate decay.
    batch = tf.Variable(0)
    # Decay once per epoch, using an exponential schedule starting at 0.01.
    learning_rate = tf.train.exponential_decay(
            0.01,                # Base learning rate.
            batch * batch_size,  # Current index into the dataset.
            train_size,          # Decay step.
            0.95,                # Decay rate.
            staircase=True)
    # Use simple momentum for the optimization.
    optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(loss, global_step=batch)

    # Predictions for the current training minibatch.
    train_prediction = tf.nn.softmax(logits)

    # Predictions for the test and validation, which we'll compute less often.
    eval_prediction = tf.nn.softmax(model(eval_data))

    # Small utility function to evaluate a dataset by feeding batches of data to
    # {eval_data} and pulling the results from {eval_predictions}.
    # Saves memory and enables this to run on smaller GPUs.
    def eval_in_batches(data, sess):
        """Get all predictions for a dataset by running it in small batches."""
        size = data.shape[0]
        if size < EVAL_BATCH_SIZE:
            raise ValueError("batch size for evals larger than dataset: %d" % size)
        predictions = np.ndarray(shape=(size, num_labels), dtype=np.float32)
        for begin in xrange(0, size, EVAL_BATCH_SIZE):
            end = begin + EVAL_BATCH_SIZE
            if end <= size:
                predictions[begin:end, :] = sess.run(
                        eval_prediction,
                        feed_dict={eval_data: data[begin:end, ...]})
            else:
                batch_predictions = sess.run(
                        eval_prediction,
                        feed_dict={eval_data: data[-EVAL_BATCH_SIZE:, ...]})
                predictions[begin:, :] = batch_predictions[begin - size:, :]
        return predictions

    # Create a local session to run the training.
    start_time = time.time()
    with tf.Session() as sess:
        # Run all the initializers to prepare the trainable parameters.
        tf.initialize_all_variables().run()
        print('Initialized!')
        # Loop through training steps.
        for step in xrange(int(NUM_EPOCHS * train_size) // batch_size):
            # Compute the offset of the current minibatch in the data.
            # Note that we could use better randomization across epochs.
            offset = (step * batch_size) % (train_size - batch_size)
            batch_data = train_dataset[offset:(offset + batch_size), ...]
            batch_labels = train_labels[offset:(offset + batch_size)]
            # This dictionary maps the batch data (as a numpy array) to the
            # node in the graph is should be fed to.
            feed_dict = {train_data_node: batch_data,
                                     train_labels_node: batch_labels}
            # Run the graph and fetch some of the nodes.
            _, l, lr, predictions = sess.run(
                    [optimizer, loss, learning_rate, train_prediction],
                    feed_dict=feed_dict)
            if step % EVAL_FREQUENCY == 0:
                elapsed_time = time.time() - start_time
                start_time = time.time()
                print('Step %d (epoch %.2f), %.1f ms' %
                            (step, float(step) * batch_size / train_size,
                             1000 * elapsed_time / EVAL_FREQUENCY))
                print('Minibatch loss: %.3f, learning rate: %.6f' % (l, lr))
                print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
                print('Validation accuracy: %.1f%%' % accuracy(
                        eval_in_batches(valid_dataset, sess), valid_labels))
                sys.stdout.flush()
        # Finally print the result!
        test_error = accuracy(eval_in_batches(test_dataset, sess), test_labels)
        print('Test accuracy: %.1f%%' % test_error)
        
lenet()

Initialized!
Step 0 (epoch 0.00), 6.6 ms
Minibatch loss: 9.657, learning rate: 0.010000
Minibatch accuracy: 18.8%
Validation accuracy: 18.9%
Step 100 (epoch 0.03), 348.3 ms
Minibatch loss: 3.597, learning rate: 0.010000
Minibatch accuracy: 90.6%
Validation accuracy: 80.9%
Step 200 (epoch 0.06), 349.7 ms
Minibatch loss: 3.588, learning rate: 0.010000
Minibatch accuracy: 82.8%
Validation accuracy: 83.1%
Step 300 (epoch 0.10), 350.8 ms
Minibatch loss: 3.895, learning rate: 0.010000
Minibatch accuracy: 68.8%
Validation accuracy: 83.8%
Step 400 (epoch 0.13), 351.7 ms
Minibatch loss: 3.810, learning rate: 0.010000
Minibatch accuracy: 82.8%
Validation accuracy: 85.1%
Step 500 (epoch 0.16), 362.1 ms
Minibatch loss: 3.351, learning rate: 0.010000
Minibatch accuracy: 90.6%
Validation accuracy: 85.5%
Step 600 (epoch 0.19), 362.8 ms
Minibatch loss: 3.475, learning rate: 0.010000
Minibatch accuracy: 82.8%
Validation accuracy: 86.1%
Step 700 (epoch 0.22), 350.8 ms
Minibatch loss: 3.278, learning rat

## Accuracy - 97.2%
### This could be increased with more epochs. But due to time constraints, I did with only 10 epochs